# Concordia 教程（中文离线版）

本 Notebook 是 `examples/tutorial.ipynb` 的中文化副本，默认以“离线模式”运行：
- 不调用外部 LLM 接口（可手动开启）。
- 句向量嵌入器使用简易占位实现（可切换为 sentence-transformers）。
- 不修改任何原始代码与示例，仅在 `architecture/` 下提供副本以便学习与演示。


In [ ]:
# 基础导入（尽量与原教程保持一致）
import numpy as np
from IPython import display
from concordia.language_model import utils as language_model_utils
from concordia.prefabs.simulation import generic as simulation
import concordia.prefabs.entity as entity_prefabs
import concordia.prefabs.game_master as game_master_prefabs

from concordia.typing import prefab as prefab_lib
from concordia.utils import helper_functions


## 语言模型选择（默认关闭）
- 若需要真实 LLM，请填入 `API_KEY` 并将 `DISABLE_LANGUAGE_MODEL` 设为 `False`。
- 可选 `API_TYPE`：'openai' / 'together_ai' / 'vertex' / 'azure' / 'ollama' 等，视本地/云端环境而定。


In [ ]:
# 中文化默认参数：离线运行
API_KEY = ''  # 如需在线调用，请填入你的 API Key
API_TYPE = 'openai'  # 例如 'together_ai'、'openai' 等
MODEL_NAME = 'gpt-4o'  # 示例：Together 可用 'google/gemma-3-27b-it'
DISABLE_LANGUAGE_MODEL = True  # 默认为 True：完全离线


In [ ]:
# 根据选择初始化语言模型。离线时会使用 NoLanguageModel。
if not DISABLE_LANGUAGE_MODEL and not API_KEY:
    raise ValueError('需要提供 API_KEY 或将 DISABLE_LANGUAGE_MODEL 设为 True')

model = language_model_utils.language_model_setup(
    api_type=API_TYPE,
    model_name=MODEL_NAME,
    api_key=API_KEY,
    disable_language_model=DISABLE_LANGUAGE_MODEL,
)
model


## 设置句向量嵌入器（离线占位或 sentence-transformers）
- 离线：使用 `np.ones(3)` 作为固定维度嵌入向量，仅用于演示。
- 在线：使用 `sentence-transformers`（首次需要联网下载模型）。


In [ ]:
if DISABLE_LANGUAGE_MODEL:
    embedder = lambda _: np.ones(3)
else:
    import sentence_transformers
    st_model = sentence_transformers.SentenceTransformer(
        'sentence-transformers/all-mpnet-base-v2'
    )
    embedder = lambda x: st_model.encode(x, show_progress_bar=False)
embedder


## 载入可用的 Prefab（实体与 Game Master）


In [ ]:
prefabs = {
    **helper_functions.get_package_classes(entity_prefabs),
    **helper_functions.get_package_classes(game_master_prefabs),
}
display.display(display.Markdown(helper_functions.print_pretty_prefabs(prefabs)))


## 配置实例（示例：英国内战时期的对话场景）
- 两位历史人物作为实体（中文名），一个通用规则的 Game Master，以及一个初始化规则。


In [ ]:
instances = [
    prefab_lib.InstanceConfig(
        prefab='basic__Entity',
        role=prefab_lib.Role.ENTITY,
        params={
            'name': '奥利弗·克伦威尔',
            'goal': '成为英格兰护国公',
        },
    ),
    prefab_lib.InstanceConfig(
        prefab='basic__Entity',
        role=prefab_lib.Role.ENTITY,
        params={
            'name': '查理一世',
            'goal': '避免被以叛国罪处决',
        },
    ),
    prefab_lib.InstanceConfig(
        prefab='generic__GameMaster',
        role=prefab_lib.Role.GAME_MASTER,
        params={
            'name': '通用规则',
            'extra_event_resolution_steps': '',
        },
    ),
    prefab_lib.InstanceConfig(
        prefab='formative_memories_initializer__GameMaster',
        role=prefab_lib.Role.INITIALIZER,
        params={
            'name': '初始化规则',
            'next_game_master_name': '通用规则',
            'shared_memories': [
                '1646 年，国王被议会军俘获。',
                '查理一世被以叛国罪审判并被判有罪。',
            ],
        },
    ),
]
instances


In [ ]:
config = prefab_lib.Config(
    default_premise='今天是 1649 年 1 月 29 日。',
    default_max_steps=3,
    prefabs=prefabs,
    instances=instances,
)
config


## 初始化并运行仿真


In [ ]:
runnable_simulation = simulation.Simulation(
    config=config,
    model=model,
    embedder=embedder,
)
raw_log = []
results_log = runnable_simulation.play(max_steps=3, raw_log=raw_log)
len(results_log), len(raw_log)


## 查看结果（示例）


In [ ]:
# 打印最后一步概要（若存在）
if results_log:
    display.display(results_log[-1])
else:
    print('没有结果条目。')
